In [1]:
# import calibration_tools
import os
import torchvision.datasets as dset
import torchvision.transforms as trn
import torchvision.transforms.functional as trnF
import torchvision.models as models
import torch.utils.model_zoo as model_zoo
from torchvision import models
from __future__ import print_function
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import torchvision
import numpy as np
import time
from scipy import misc
import pretrainedmodels

In [2]:
ROOT = "/scratch/as14770/CV-FinalProject"

PATH_CONFIDENCE_BASE_IN = ROOT + "/confidence_Base_In.txt"
PATH_CONFIDENCE_BASE_OUT = ROOT + "/confidence_Base_Out.txt"
PATH_CONFIDENCE_OUR_IN = ROOT + "/confidence_Our_In.txt"
PATH_CONFIDENCE_OUR_OUT = ROOT + "/confidence_Our_Out.txt"


epsilon = 0.0014   # perturbation 21 evenly spaced numbers b/w 0 and 0.004 
temperature = 1000  # temperature scaling [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]
criterion = nn.CrossEntropyLoss()
CUDA_DEVICE = 0   # set to 1 to use gpu
# TODO: start and end also may need to be tuned

In [3]:
def tpr95(name):
    #calculate the falsepositive error when tpr is 95%
    # calculate baseline
#     T = 1
    indistr = np.loadtxt(PATH_CONFIDENCE_BASE_IN, delimiter=',')
    outdistr = np.loadtxt(PATH_CONFIDENCE_BASE_OUT, delimiter=',')
#     if name == "CIFAR-10": 
#       start = 0.1
#       end = 1 
#     if name == "CIFAR-100": 
    start = 0.001
    end = 1 
    gap = (end- start)/100000
    #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
    Y1 = outdistr[:, 2]
    X1 = indistr[:, 2]
    total = 0.0
    fpr = 0.0
    for delta in np.arange(start, end, gap):
        tpr = np.sum(np.sum(X1 >= delta)) / np.float(len(X1))
        error2 = np.sum(np.sum(Y1 > delta)) / np.float(len(Y1))
        if tpr <= 0.9505 and tpr >= 0.9495:
            fpr += error2
            total += 1
    fprBase = fpr/total

    # calculate our algorithm
#     T = 1000
    indistr = np.loadtxt(PATH_CONFIDENCE_OUR_IN, delimiter=',')
    outdistr = np.loadtxt(PATH_CONFIDENCE_OUR_OUT, delimiter=',')
#     if name == "CIFAR-10": 
#       start = 0.1
#       end = 0.12 
#     if name == "CIFAR-100": 
#       start = 0.01
#       end = 0.0104  
    start = 0.001
    end = 0.0011
    gap = (end- start)/100000
    #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
    Y1 = outdistr[:, 2]
    X1 = indistr[:, 2]
    total = 0.0
    fpr = 0.0
    for delta in np.arange(start, end, gap):
        tpr = np.sum(np.sum(X1 >= delta)) / np.float(len(X1))
        error2 = np.sum(np.sum(Y1 > delta)) / np.float(len(Y1))
        if tpr <= 0.9505 and tpr >= 0.9495:
            fpr += error2
            total += 1
    fprNew = fpr/total
            
    return fprBase, fprNew

def auroc(name):
    #calculate the AUROC
    # calculate baseline
#     T = 1
    indistr = np.loadtxt(PATH_CONFIDENCE_BASE_IN, delimiter=',')
    outdistr = np.loadtxt(PATH_CONFIDENCE_BASE_OUT, delimiter=',')
#     if name == "CIFAR-10": 
#       start = 0.1
#       end = 1 
#     if name == "CIFAR-100": 
    start = 0.001
    end = 1 
    gap = (end- start)/100000
    #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
    Y1 = indistr[:, 2]
    X1 = outdistr[:, 2]
    aurocBase = 0.0
    fprTemp = 1.0
    for delta in np.arange(start, end, gap):
        tpr = np.sum(np.sum(X1 >= delta)) / np.float(len(X1))
        fpr = np.sum(np.sum(Y1 > delta)) / np.float(len(Y1))
        aurocBase += (-fpr+fprTemp)*tpr
        fprTemp = fpr
    aurocBase += fpr * tpr
    # calculate our algorithm
#     T = 1000
    indistr = np.loadtxt(PATH_CONFIDENCE_OUR_IN, delimiter=',')
    outdistr = np.loadtxt(PATH_CONFIDENCE_OUR_OUT, delimiter=',')
#     if name == "CIFAR-10": 
#       start = 0.1
#       end = 0.12 
#     if name == "CIFAR-100": 
#       start = 0.01
#       end = 0.0104   
    start = 0.001
    end = 0.0011
    gap = (end- start)/100000
    #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
    Y1 = outdistr[:, 2]
    X1 = indistr[:, 2]
    aurocNew = 0.0
    fprTemp = 1.0
    for delta in np.arange(start, end, gap):
        tpr = np.sum(np.sum(X1 >= delta)) / np.float(len(X1))
        fpr = np.sum(np.sum(Y1 >= delta)) / np.float(len(Y1))
        aurocNew += (-fpr+fprTemp)*tpr
        fprTemp = fpr
    aurocNew += fpr * tpr
    return aurocBase, aurocNew

def auprIn(name):
    #calculate the AUPR
    # calculate baseline
#     T = 1
    indistr = np.loadtxt(PATH_CONFIDENCE_BASE_IN, delimiter=',')
    outdistr = np.loadtxt(PATH_CONFIDENCE_BASE_OUT, delimiter=',')
#     if name == "CIFAR-10": 
#       start = 0.1
#       end = 1 
#     if name == "CIFAR-100": 
    start = 0.001
    end = 1   
    gap = (end- start)/100000
    precisionVec = []
    recallVec = []
        #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
    Y1 = outdistr[:, 2]
    X1 = indistr[:, 2]
    auprBase = 0.0
    recallTemp = 1.0
    for delta in np.arange(start, end, gap):
        tp = np.sum(np.sum(X1 >= delta)) / np.float(len(X1))
        fp = np.sum(np.sum(Y1 >= delta)) / np.float(len(Y1))
        if tp + fp == 0: continue
        precision = tp / (tp + fp)
        recall = tp
        precisionVec.append(precision)
        recallVec.append(recall)
        auprBase += (recallTemp-recall)*precision
        recallTemp = recall
    auprBase += recall * precision
    #print(recall, precision)

    # calculate our algorithm
#     T = 1000
    inddistr = np.loadtxt(PATH_CONFIDENCE_OUR_IN, delimiter=',')
    outdistr = np.loadtxt(PATH_CONFIDENCE_OUR_OUT, delimiter=',')
#     if name == "CIFAR-10": 
#       start = 0.1
#       end = 0.12 
#     if name == "CIFAR-100": 
#       start = 0.01
#       end = 0.0104   
    start = 0.001
    end = 0.0011
    gap = (end- start)/100000
    #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
    Y1 = outdistr[:, 2]
    X1 = indistr[:, 2]
    auprNew = 0.0
    recallTemp = 1.0
    for delta in np.arange(start, end, gap):
        tp = np.sum(np.sum(X1 >= delta)) / np.float(len(X1))
        fp = np.sum(np.sum(Y1 >= delta)) / np.float(len(Y1))
        if tp + fp == 0: continue
        precision = tp / (tp + fp)
        recall = tp
        #precisionVec.append(precision)
        #recallVec.append(recall)
        auprNew += (recallTemp-recall)*precision
        recallTemp = recall
    auprNew += recall * precision
    return auprBase, auprNew

def auprOut(name):
    #calculate the AUPR
    # calculate baseline
#     T = 1
    indistr = np.loadtxt(PATH_CONFIDENCE_BASE_IN, delimiter=',')
    outdistr = np.loadtxt(PATH_CONFIDENCE_BASE_OUT, delimiter=',')
#     if name == "CIFAR-10": 
#       start = 0.1
#       end = 1 
#     if name == "CIFAR-100": 
    start = 0.001
    end = 1   
    gap = (end- start)/100000
    Y1 = outdistr[:, 2]
    X1 = indistr[:, 2]
    auprBase = 0.0
    recallTemp = 1.0
    for delta in np.arange(end, start, -gap):
        fp = np.sum(np.sum(X1 < delta)) / np.float(len(X1))
        tp = np.sum(np.sum(Y1 < delta)) / np.float(len(Y1))
        if tp + fp == 0: break
        precision = tp / (tp + fp)
        recall = tp
        auprBase += (recallTemp-recall)*precision
        recallTemp = recall
    auprBase += recall * precision
        
    
    # calculate our algorithm
#     T = 1000
    indistr = np.loadtxt(PATH_CONFIDENCE_OUR_IN, delimiter=',')
    outdistr = np.loadtxt(PATH_CONFIDENCE_OUR_OUT, delimiter=',')
#     if name == "CIFAR-10": 
#       start = 0.1
#       end = 0.12 
#     if name == "CIFAR-100": 
#       start = 0.01
#       end = 0.0104
    start = 0.001
    end = 0.0011
    gap = (end- start)/100000
    #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
    Y1 = outdistr[:, 2]
    X1 = indistr[:, 2]
    auprNew = 0.0
    recallTemp = 1.0
    for delta in np.arange(end, start, -gap):
        fp = np.sum(np.sum(X1 < delta)) / np.float(len(X1))
        tp = np.sum(np.sum(Y1 < delta)) / np.float(len(Y1))
        if tp + fp == 0: break
        precision = tp / (tp + fp)
        recall = tp
        auprNew += (recallTemp-recall)*precision
        recallTemp = recall
    auprNew += recall * precision
    return auprBase, auprNew



def detection(name):
    #calculate the minimum detection error
    # calculate baseline
#     T = 1
    indistr = np.loadtxt(PATH_CONFIDENCE_BASE_IN, delimiter=',')
    outdistr = np.loadtxt(PATH_CONFIDENCE_BASE_OUT, delimiter=',')
#     if name == "CIFAR-10": 
#       start = 0.1
#       end = 1 
#     if name == "CIFAR-100": 
    start = 0.001
    end = 1    
    gap = (end- start)/100000
    #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
    Y1 = outdistr[:, 2]
    X1 = indistr[:, 2]
    errorBase = 1.0
    for delta in np.arange(start, end, gap):
        tpr = np.sum(np.sum(X1 < delta)) / np.float(len(X1))
        error2 = np.sum(np.sum(Y1 > delta)) / np.float(len(Y1))
        errorBase = np.minimum(errorBase, (tpr+error2)/2.0)

    # calculate our algorithm
#     T = 1000
    indistr = np.loadtxt(PATH_CONFIDENCE_OUR_IN, delimiter=',')
    outdistr = np.loadtxt(PATH_CONFIDENCE_OUR_OUT, delimiter=',')
#     if name == "CIFAR-10": 
#       start = 0.1
#       end = 0.12 
#     if name == "CIFAR-100": 
#       start = 0.01
#       end = 0.0104 
    start = 0.001
    end = 0.0011
    gap = (end- start)/100000
    #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
    Y1 = outdistr[:, 2]
    X1 = indistr[:, 2]
    errorNew = 1.0
    for delta in np.arange(start, end, gap):
        tpr = np.sum(np.sum(X1 < delta)) / np.float(len(X1))
        error2 = np.sum(np.sum(Y1 > delta)) / np.float(len(Y1))
        errorNew = np.minimum(errorNew, (tpr+error2)/2.0)
            
    return errorBase, errorNew


def metric(nn):
    indis = "Imagenet-1k"
    fprBase, fprNew = tpr95(indis)
    errorBase, errorNew = detection(indis)
    aurocBase, aurocNew = auroc(indis)
    auprinBase, auprinNew = auprIn(indis)
    auproutBase, auproutNew = auprOut(indis)
    print("{:31}{:>22}".format("Neural network architecture:", nn))
    print("{:31}{:>22}".format("In-distribution dataset:", indis))
    print("{:31}{:>22}".format("Out-of-distribution dataset:", "IMAGENET-O"))
    print("")
    print("{:>34}{:>19}".format("Baseline", "Our Method"))
    print("{:20}{:13.1f}%{:>18.1f}% ".format("FPR at TPR 95%:",fprBase*100, fprNew*100))
    print("{:20}{:13.1f}%{:>18.1f}%".format("Detection error:",errorBase*100, errorNew*100))
    print("{:20}{:13.1f}%{:>18.1f}%".format("AUROC:",aurocBase*100, aurocNew*100))
    print("{:20}{:13.1f}%{:>18.1f}%".format("AUPR In:",auprinBase*100, auprinNew*100))
    print("{:20}{:13.1f}%{:>18.1f}%".format("AUPR Out:",auproutBase*100, auproutNew*100))

In [4]:
def testData(net, criterion, CUDA_DEVICE, testloader10, testloader, noiseMagnitude1, temper):
    t0 = time.time()
    f1 = open(PATH_CONFIDENCE_BASE_IN, 'w+')
    f2 = open(PATH_CONFIDENCE_BASE_OUT, 'w+')
    g1 = open(PATH_CONFIDENCE_OUR_IN, 'w+')
    g2 = open(PATH_CONFIDENCE_OUR_OUT, 'w+')
    N = 10000
    print("Processing in-distribution images")
########################################In-distribution###########################################
    for j, data in enumerate(testloader10):
        if j<1000: continue
        images, _ = data
        
        inputs = Variable(images.cuda(CUDA_DEVICE), requires_grad = True)
        outputs = net(inputs)
        

        # Calculating the confidence of the output, no perturbation added here, no temperature scaling used
        nnOutputs = outputs.data.cpu()
        nnOutputs = nnOutputs.numpy()
        nnOutputs = nnOutputs[0]
        nnOutputs = nnOutputs - np.max(nnOutputs)
        nnOutputs = np.exp(nnOutputs)/np.sum(np.exp(nnOutputs))
#         print(temper, noiseMagnitude1, np.max(nnOutputs))
        f1.write("{}, {}, {}\n".format(temper, noiseMagnitude1, np.max(nnOutputs)))
        
        # Using temperature scaling
        outputs = outputs / temper
	
        # Calculating the perturbation we need to add, that is,
        # the sign of gradient of cross entropy loss w.r.t. input
        maxIndexTemp = np.argmax(nnOutputs)
        labels = Variable(torch.LongTensor([maxIndexTemp]).cuda(CUDA_DEVICE))
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Normalizing the gradient to binary in {0, 1}
        gradient =  torch.ge(inputs.grad.data, 0)
        gradient = (gradient.float() - 0.5) * 2
        # Normalizing the gradient to the same space of image  # std: [0.229, 0.224, 0.225]
        gradient[0][0] = (gradient[0][0] )/0.229
        gradient[0][1] = (gradient[0][1] )/0.224
        gradient[0][2] = (gradient[0][2])/0.225
        # Adding small perturbations to images
        tempInputs = torch.add(inputs.data,  -noiseMagnitude1, gradient)
        outputs = net(Variable(tempInputs))
        outputs = outputs / temper
        # Calculating the confidence after adding perturbations
        nnOutputs = outputs.data.cpu()
        nnOutputs = nnOutputs.numpy()
        nnOutputs = nnOutputs[0]
        nnOutputs = nnOutputs - np.max(nnOutputs)
        nnOutputs = np.exp(nnOutputs)/np.sum(np.exp(nnOutputs))
        g1.write("{}, {}, {}\n".format(temper, noiseMagnitude1, np.max(nnOutputs)))
        if j % 100 == 99:
            print("{:4}/{:4} images processed, {:.1f} seconds used.".format(j+1-1000, N-1000, time.time()-t0))
            t0 = time.time()
        
        if j == N - 1: break


    t0 = time.time()
    print("Processing out-of-distribution images")
###################################Out-of-Distributions#####################################
    for j, data in enumerate(testloader):
        if j<1000: continue
        images, _ = data
    
        inputs = Variable(images.cuda(CUDA_DEVICE), requires_grad = True)
        outputs = net(inputs)


        # Calculating the confidence of the output, no perturbation added here
        nnOutputs = outputs.data.cpu()
        nnOutputs = nnOutputs.numpy()
        nnOutputs = nnOutputs[0]
        nnOutputs = nnOutputs - np.max(nnOutputs)
        nnOutputs = np.exp(nnOutputs)/np.sum(np.exp(nnOutputs))
#         print(temper, noiseMagnitude1, np.max(nnOutputs))
        f2.write("{}, {}, {}\n".format(temper, noiseMagnitude1, np.max(nnOutputs)))
        
        # Using temperature scaling
        outputs = outputs / temper
  
  
        # Calculating the perturbation we need to add, that is,
        # the sign of gradient of cross entropy loss w.r.t. input
        maxIndexTemp = np.argmax(nnOutputs)
        labels = Variable(torch.LongTensor([maxIndexTemp]).cuda(CUDA_DEVICE))
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Normalizing the gradient to binary in {0, 1}
        gradient =  (torch.ge(inputs.grad.data, 0))
        gradient = (gradient.float() - 0.5) * 2
        # Normalizing the gradient to the same space of image # std: [0.229, 0.224, 0.225]
        gradient[0][0] = (gradient[0][0] )/0.229
        gradient[0][1] = (gradient[0][1] )/0.224
        gradient[0][2] = (gradient[0][2])/0.225
        # Adding small perturbations to images
        tempInputs = torch.add(inputs.data,  -noiseMagnitude1, gradient)
        outputs = net(Variable(tempInputs))
        outputs = outputs / temper
        # Calculating the confidence after adding perturbations
        nnOutputs = outputs.data.cpu()
        nnOutputs = nnOutputs.numpy()
        nnOutputs = nnOutputs[0]
        nnOutputs = nnOutputs - np.max(nnOutputs)
        nnOutputs = np.exp(nnOutputs)/np.sum(np.exp(nnOutputs))
        g2.write("{}, {}, {}\n".format(temper, noiseMagnitude1, np.max(nnOutputs)))
        if j % 100 == 99:
            print("{:4}/{:4} images processed, {:.1f} seconds used.".format(j+1-1000, N-1000, time.time()-t0))
            t0 = time.time()

        if j== N-1: break

In [5]:
PATH_TO_IMAGENET_O = "/scratch/as14770/CV-FinalProject/DATA/imagenet-o"
PATH_TO_IMAGENET_VAL = "/scratch/as14770/CV-FinalProject/DATA/imagenet1k-val"
TORCH_HOME_DIR = "/scratch/as14770/CV-FinalProject/.cvfpmodels/"

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

test_transform = trn.Compose(
    [trn.Resize(256), trn.CenterCrop(224), trn.ToTensor(), trn.Normalize(mean, std)])

noes = dset.ImageFolder(root=PATH_TO_IMAGENET_O, transform=test_transform)
noe_loader = torch.utils.data.DataLoader(noes, batch_size=1, shuffle=False,
                                         num_workers=4, pin_memory=True)

imagenet_o_folder = "/scratch/as14770/CV-FinalProject/imagenet-o-symlink/"

def create_symlinks_to_imagenet(imagenet_folder, folder_to_scan):
    if not os.path.exists(imagenet_folder):
        os.makedirs(imagenet_folder)
        folders_of_interest = os.listdir(folder_to_scan)
        path_prefix = PATH_TO_IMAGENET_VAL 
        for folder in folders_of_interest:
            os.symlink(path_prefix + folder, imagenet_folder+folder, target_is_directory=True)

create_symlinks_to_imagenet(imagenet_o_folder, PATH_TO_IMAGENET_O)

val_examples_imagenet_o = dset.ImageFolder(root=imagenet_o_folder, transform=test_transform)
val_loader_imagenet_o = torch.utils.data.DataLoader(val_examples_imagenet_o, batch_size=1, shuffle=False,
                                         num_workers=4, pin_memory=True)

val_imagenet = dset.ImageNet(root=PATH_TO_IMAGENET_VAL, split = 'val', transform=test_transform)
val_imagenet_loader = torch.utils.data.DataLoader(val_imagenet, batch_size=1, shuffle=False,
                                         num_workers=4, pin_memory=True)

concat = lambda x: np.concatenate(x, axis=0)
to_np = lambda x: x.data.to('cpu').numpy()

os.environ["TORCH_HOME"] = TORCH_HOME_DIR
#print(os.environ["TORCH_HOME"])
torch.hub.set_dir(TORCH_HOME_DIR)
models_to_test = [
    ("pytorch/vision", "alexnet"),
    ("pytorch/vision", "squeezenet1_1"),
    ("pytorch/vision", "vgg16"),
    ("pytorch/vision", "vgg19"),
    ('pytorch/vision', "vgg19_bn"),
    ('pytorch/vision', "densenet121"),
    ('pytorch/vision', "resnet50"),
    ('pytorch/vision', "resnet101"),
    ('pytorch/vision', "resnet152"),
    ('pytorch/vision', "wide_resnet50_2"),
    ('pytorch/vision', "resnext101_32x8d"),
    ('pytorch/vision', "resnext50_32x4d"),
    ('facebookresearch/WSL-Images', "resnext101_32x8d_wsl"),
    ('facebookresearch/WSL-Images', "resnext101_32x16d_wsl"),
    ('facebookresearch/WSL-Images', "resnext101_32x32d_wsl"),
    ("pretrained", "dpn68"),
    ("pretrained", "dpn98"),
    ("pretrained", "se_resnet101"),
    ("pretrained", "se_resnet152"),
    ("pretrained", "resnext101_32x4d"),
    ("pretrained", "se_resnext101_32x4d"),
    ("facebookresearch/deit:main", "deit_base_patch16_224"),
    ("facebookresearch/deit:main", "deit_small_patch16_224"),
    ("facebookresearch/deit:main", "deit_tiny_patch16_224"),
]

for net_params in models_to_test:
    if net_params[0] == "pytorch/vision":
        net = torch.hub.load('pytorch/vision:v0.10.0', net_params[1], pretrained=True)
#         net = models.alexnet(pretrained=True)
    elif "facebookresearch/deit" in net_params[0]:
        net = torch.hub.load(net_params[0], net_params[1], pretrained=True)
    elif net_params[0] == "pretrained":
        net = pretrainedmodels.__dict__[net_params[1]](num_classes=1000, pretrained='imagenet')
    else:
        net = torch.hub.load('/scratch/as14770/CV-FinalProject/facebookresearch_WSL-Images_master', net_params[1], source='local')
    print(net_params[1], '\n')
    net.cuda()
    net.eval()
    testData(net, criterion, CUDA_DEVICE, val_loader_imagenet_o, noe_loader, epsilon, temperature) 
    metric(net_params[1])
    print("\n\n\n")

Using cache found in /scratch/as14770/CV-FinalProject/.cvfpmodels/pytorch_vision_v0.10.0


alexnet 

Processing in-distribution images


/state/partition1/job-12664546/ipykernel_3000882/4219558165.py:45: UserWarning: This overload of add is deprecated:
	add(Tensor input, Number alpha, Tensor other, *, Tensor out)
Consider using one of the following signatures instead:
	add(Tensor input, Tensor other, *, Number alpha, Tensor out) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)
  tempInputs = torch.add(inputs.data,  -noiseMagnitude1, gradient)


 100/9000 images processed, 8.7 seconds used.
 200/9000 images processed, 0.7 seconds used.
 300/9000 images processed, 0.7 seconds used.
 400/9000 images processed, 0.7 seconds used.
 500/9000 images processed, 0.7 seconds used.
 600/9000 images processed, 0.8 seconds used.
 700/9000 images processed, 0.9 seconds used.
 800/9000 images processed, 0.7 seconds used.
 900/9000 images processed, 0.7 seconds used.
1000/9000 images processed, 0.7 seconds used.
1100/9000 images processed, 0.7 seconds used.
1200/9000 images processed, 1.0 seconds used.
1300/9000 images processed, 0.8 seconds used.
1400/9000 images processed, 0.8 seconds used.
1500/9000 images processed, 0.7 seconds used.
1600/9000 images processed, 0.7 seconds used.
1700/9000 images processed, 0.7 seconds used.
1800/9000 images processed, 0.7 seconds used.
1900/9000 images processed, 0.9 seconds used.
2000/9000 images processed, 0.7 seconds used.
2100/9000 images processed, 0.7 seconds used.
2200/9000 images processed, 0.7 se

Using cache found in /scratch/as14770/CV-FinalProject/.cvfpmodels/pytorch_vision_v0.10.0


...

deit_base_patch16_224 

Processing in-distribution images
 100/9000 images processed, 8.2 seconds used.
 200/9000 images processed, 3.2 seconds used.
 300/9000 images processed, 3.2 seconds used.
 400/9000 images processed, 3.1 seconds used.
 500/9000 images processed, 3.2 seconds used.
 600/9000 images processed, 3.1 seconds used.
 700/9000 images processed, 3.1 seconds used.
 800/9000 images processed, 3.2 seconds used.
 900/9000 images processed, 3.1 seconds used.
1000/9000 images processed, 3.1 seconds used.
1100/9000 images processed, 3.2 seconds used.
1200/9000 images processed, 3.1 seconds used.
1300/9000 images processed, 3.2 seconds used.
1400/9000 images processed, 3.2 seconds used.
1500/9000 images processed, 3.1 seconds used.
1600/9000 images processed, 3.1 seconds used.
1700/9000 images processed, 3.1 seconds used.
1800/9000 images processed, 3.1 seconds used.
1900/9000 images processed, 3.2 seconds used.
2000/9000 images processed, 3.1 seconds used.
2100/9000 images proce

Using cache found in /scratch/as14770/CV-FinalProject/.cvfpmodels/facebookresearch_deit_main


deit_small_patch16_224 

Processing in-distribution images
 100/9000 images processed, 8.0 seconds used.
 200/9000 images processed, 3.1 seconds used.
 300/9000 images processed, 3.2 seconds used.
 400/9000 images processed, 3.2 seconds used.
 500/9000 images processed, 3.1 seconds used.
 600/9000 images processed, 3.1 seconds used.
 700/9000 images processed, 3.2 seconds used.
 800/9000 images processed, 3.2 seconds used.
 900/9000 images processed, 3.1 seconds used.
1000/9000 images processed, 3.1 seconds used.
1100/9000 images processed, 3.1 seconds used.
1200/9000 images processed, 3.2 seconds used.
1300/9000 images processed, 3.1 seconds used.
1400/9000 images processed, 3.1 seconds used.
1500/9000 images processed, 3.1 seconds used.
1600/9000 images processed, 3.2 seconds used.
1700/9000 images processed, 3.2 seconds used.
1800/9000 images processed, 3.1 seconds used.
1900/9000 images processed, 3.2 seconds used.
2000/9000 images processed, 3.1 seconds used.
2100/9000 images proc

Using cache found in /scratch/as14770/CV-FinalProject/.cvfpmodels/facebookresearch_deit_main


deit_tiny_patch16_224 

Processing in-distribution images
 100/9000 images processed, 6.8 seconds used.
 200/9000 images processed, 3.1 seconds used.
 300/9000 images processed, 3.1 seconds used.
 400/9000 images processed, 3.1 seconds used.
 500/9000 images processed, 3.1 seconds used.
 600/9000 images processed, 3.1 seconds used.
 700/9000 images processed, 3.1 seconds used.
 800/9000 images processed, 3.1 seconds used.
 900/9000 images processed, 3.1 seconds used.
1000/9000 images processed, 3.1 seconds used.
1100/9000 images processed, 3.1 seconds used.
1200/9000 images processed, 3.1 seconds used.
1300/9000 images processed, 3.1 seconds used.
1400/9000 images processed, 3.1 seconds used.
1500/9000 images processed, 3.1 seconds used.
1600/9000 images processed, 3.1 seconds used.
1700/9000 images processed, 3.1 seconds used.
1800/9000 images processed, 3.1 seconds used.
1900/9000 images processed, 3.1 seconds used.
2000/9000 images processed, 3.1 seconds used.
2100/9000 images proce